## Conditional Graph
The objective of this graph is to:
1. implement conditional logic to route the flow of data to different nodes.
2. use `START` and `END` nodes to manage entry and exit points explicitly.
3. design multiple nodes to perform different operations (addition, subtraction)
4. create a router node to handle the decision making and control the flow of the graph

Graph output:
$$START -> router -> addition operation -> add_{node} -> END$$
or 
$$START -> router -> subtract operation -> subtract_{node} -> END$$

Based on the operation given the decision node (`router`) is going to termine what path to take.
The goal of this graph is to use `add_conditional_edges()`.

In [ ]:
from typing import TypeDict
from langgraph.graph import StateGraph, START, END

In [ ]:
# define agent state
# GOAL: pass in two numbers and two operations
class AgentState(TypeDict):
    num1: int
    operation: str
    num2: int
    result: int  # result of the operation

# define the node
def adder(state: AgentState) -> AgentState:
    """This node addes the two numbers"""

    state["result"] = state["num1"] + state["num2"]
    return state

def subtracter(state: AgentState) -> AgentState:
    """This node subtracts the two numbers"""

    state["result"] = state["num1"] - state["num2"]
    return state

# this node is the decision maker based on the agent state schema (router)
def decide_next_node(state: AgentState) -> AgentState:
    """This node will select the next node of the graph"""

    # define the condition to run either the add node or subtract node
    if state["operation"] == "+":
        # if the operation is a +, we need to return the edge definition
        return "addition_operation"
    elif state["operation"] == "-":
        return "subtration_operation"
    
    # this does not return the state, only the graph's edge (so when adding the node we will need to define a passthrough function)
    
# define the graph, nodes, and edges
workflow = StateGraph(AgentState)
workflow.add_node("add", adder)
workflow.add_node('subtract', subtracter)
workflow.add_node("router", lambda state:state) # you are inputing and output the state

workflow.add_edge(START, "route")
workflow.add_conditional_edges(
    "router",  # src node
    decide_next_node, # path
    {  # path map
        # edge: node
        "addition_operation": "add",
        "subtraction_operation": "subtract"
    }
)
workflow.add_edge("add", END)
workflow.add_edge("subtract", END)

application = workflow.compile()


In [ ]:
# extra step, but this visualizes the graph we created in LangGraph
from IPython.display import Image, display
display(Image(application.get_graph().draw_mermaid_png()))